In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# Locate the Project 1 directory to this notebook's working directory
PROJECT_DIR = Path.cwd()
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
OUT_DIR = os.path.join(PROJECT_DIR, 'output')
for d in [DATA_DIR, OUT_DIR]:
    os.makedirs(d, exist_ok=True)

In [3]:
merged = pd.read_csv('data/merged_data.csv', index_col=0, parse_dates=True)

In [4]:
#Rename the column ret by ret_sp to avoid confusion with ret_gold and ret_oil
merged.rename(columns={'ret': 'ret_sp'}, inplace=True)


In [5]:
merged

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,DGS10,ret_sp
Date,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.270020,1320.280029,1276.050049,1320.280029,1129400000,29.990000,30.400000,27.920000,27.930000,268.399994,268.399994,268.399994,268.399994,27.200001,27.400000,26.600000,27.250000,0.014231,0.0492,NaN
2001-01-03,1347.560059,1347.760010,1274.619995,1283.270020,1880700000,26.600000,29.959999,25.990000,29.629999,268.000000,268.000000,268.000000,268.000000,27.950001,28.139999,27.049999,27.230000,0.031663,0.0514,0.048884
2001-01-04,1333.339966,1350.239990,1329.140015,1347.560059,2131000000,26.969999,27.190001,26.129999,26.350000,267.299988,267.299988,267.299988,267.299988,28.200001,28.780001,27.850000,28.200001,0.012610,0.0503,-0.010609
2001-01-05,1298.349976,1334.770020,1294.949951,1333.339966,1430800000,28.670000,29.040001,27.209999,27.809999,268.000000,268.000000,268.000000,268.000000,28.000000,28.799999,27.799999,28.150000,0.015626,0.0493,-0.026593
2001-01-08,1295.859985,1298.349976,1276.290039,1298.349976,1115500000,29.840000,30.799999,29.719999,29.980000,268.000000,268.000000,268.000000,268.000000,27.350000,28.400000,27.150000,28.200001,0.009083,0.0494,-0.001920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,3909.879883,3931.500000,3884.939941,3920.780029,4837070000,21.990000,23.850000,19.690001,21.639999,1840.599976,1851.099976,1837.000000,1842.599976,58.680000,58.910000,58.080002,58.450001,0.008828,0.0115,-0.000345
2021-02-11,3916.379883,3925.989990,3890.389893,3916.399902,4590960000,21.250000,23.250000,21.110001,22.090000,1824.900024,1845.000000,1820.900024,1840.500000,58.240002,58.709999,57.840000,58.400002,0.006555,0.0116,0.001661
2021-02-12,3934.830078,3937.229980,3905.780029,3911.649902,4135060000,19.969999,22.450001,19.950001,21.600000,1821.599976,1827.500000,1809.199951,1825.000000,59.470001,59.820000,57.410000,57.939999,0.003810,0.0120,0.004700


In [ ]:
# A) MICROSTRUCTURE-BASED FEATURES:

# A.1) RELATIVE VOLUME

vol_ma = merged['Volume_S&P'].rolling(window=60, min_periods=1).mean()

# Calculate the Ratio
merged['Relative_Volume'] = merged['Volume_S&P'] / vol_ma

#A.2) TRUE PRICE RANGE

prev_close = merged['Close_S&P'].shift(1)

# Component A: High - Low
tr_a = merged['High_S&P'] - merged['Low_S&P']

# Component B: |High - Prev Close|
tr_b = (merged['High_S&P'] - prev_close).abs()

# Component C: |Low - Prev Close|
tr_c = (merged['Low_S&P'] - prev_close).abs()

# Take the Max of the three components row-by-row
merged['True_Range_S&P'] = pd.concat([tr_a, tr_b, tr_c], axis=1).max(axis=1)





In [ ]:
#B) Macro-Financial regime Features:

#B.1) log returns and basis point changes

#WE NEED TO HANDLE 4 OBSERVATIONS WERE FUTURE OIL PRICE WAS NEGATIVE (COVID LOCKDOWN) TO AVOID ISSUE WITH NEGATIVE VALUE IN THE LOGARITHM:
#4 observations is really marginal so we will forward fill the previous day values for these 4 days.

def safe_log_return(series):
    # Mask negative/zero prices to NaN
    clean_series = series.mask(series <= 0)
    # Forward fill prices (assume price stayed same as yesterday if today is invalid)
    clean_series = clean_series.ffill()
    # Calculate Log Return
    return np.log(clean_series).diff()

# Log Returns for Asset Prices
merged['ret_gold'] = safe_log_return(merged['Close_GOLD'])
merged['ret_oil'] = safe_log_return(merged['Close_OIL'])

# Basis Point Change for Yields
merged['diff_dgs10'] = safe_log_return(merged['DGS10'])

#----------------------------------

#B.2) 60-DAY ROLLING CORRELATIONS

# S&P 500 vs. GOLD (Risk-Off Indicator)
merged['Corr_SP_Gold'] = merged['ret_sp'].rolling(window=60).corr(merged['ret_gold'])

# S&P 500 vs. CRUDE OIL (Inflation/Growth Indicator)
merged['Corr_SP_Oil'] = merged['ret_sp'].rolling(window=60).corr(merged['ret_oil'])

# S&P 500 vs. 10Y YIELD (Macro-Financial Environment)
merged['Corr_SP_DGS10'] = merged['ret_sp'].rolling(window=60).corr(merged['diff_dgs10'])

#----------------------------------

#B.3) MICICROSTRUCUTRE OIL / GOLD 

#1) TRUE PRICE RANGE : oiL

prev_close = merged['Close_OIL'].shift(1)

# Component A: High - Low
tr_a = merged['High_OIL'] - merged['Low_OIL']

# Component B: |High - Prev Close|
tr_b = (merged['High_OIL'] - prev_close).abs()

# Component C: |Low - Prev Close|
tr_c = (merged['Low_OIL'] - prev_close).abs()

# Take the Max of the three components row-by-row
merged['True_Range_Oil'] = pd.concat([tr_a, tr_b, tr_c], axis=1).max(axis=1)


#2) TRUE PRICE RANGE : gold
prev_close = merged['Close_GOLD'].shift(1)

# Component A: High - Low
tr_a = merged['High_GOLD'] - merged['Low_GOLD']

# Component B: |High - Prev Close|
tr_b = (merged['High_GOLD'] - prev_close).abs()

# Component C: |Low - Prev Close|
tr_c = (merged['Low_GOLD'] - prev_close).abs()

# Take the Max of the three components row-by-row
merged['True_Range_Gold'] = pd.concat([tr_a, tr_b, tr_c], axis=1).max(axis=1)

#----------------------------------

#N.4) GARMAN-KLASS VOLATILITY ESTIMATOR FOR OIL AND GOLD
#WE NEED TO HANDLE 4 OBSERVATIONS WERE FUTURE OIL PRICE WAS NEGATIVE (COVID LOCKDOWN) TO AVOID ISSUE WITH NEGATIVE VALUE IN THE LOGARITHM:
#4 observations is really marginal so we will forward fill the previous day values for these 4 days.


def calculate_gk_volatility(high_series, low_series, close_series, open_series):
    """
    Calculates Garman-Klass volatility.
    
    Robustness Logic:
    1. Identifies rows with negative/zero prices (Invalid for Log).
    2. Sets those specific rows to NaN to prevent RuntimeWarnings.
    3. Forward Fills (ffill) the resulting Volatility to replace the NaN 
       with the previous day's valid volatility.
    """
    
    # 1. Create a mask for INVALID rows (Any price <= 0)
    # This captures the "Negative Oil Price" scenario
    bad_data_mask = (high_series <= 0) | (low_series <= 0) | (close_series <= 0) | (open_series <= 0)
    
    # 2. Create temporary Clean Series
    # We replace bad values with NaN. This prevents np.log from crashing or throwing warnings.
    h = high_series.mask(bad_data_mask)
    l = low_series.mask(bad_data_mask)
    c = close_series.mask(bad_data_mask)
    o = open_series.mask(bad_data_mask)
    
    # 3. Calculate Logs (Invalid rows will naturally become NaN)
    log_hl = np.log(h / l)
    log_co = np.log(c / o)
    
    # 4. Variance Formula
    gk_variance = 0.5 * (log_hl ** 2) - (2 * np.log(2) - 1) * (log_co ** 2)
    
    # 5. Calculate Volatility (Standard Deviation)
    # We clip at 0 to prevent tiny negative float errors, then sqrt
    gk_vol = np.sqrt(gk_variance.clip(lower=0))
    
    # 6: Forward Fill
    # Wherever we have a NaN (due to negative prices), copy the value from yesterday.
    gk_vol_filled = gk_vol.ffill()
    
    
    return gk_vol_filled

#1) OIL
merged['Vol_GK_Oil'] = calculate_gk_volatility(
    merged['High_OIL'], 
    merged['Low_OIL'], 
    merged['Close_OIL'], 
    merged['Open_OIL']
)


#2) GOLD 
merged['Vol_GK_Gold'] = calculate_gk_volatility(
    merged['High_GOLD'], 
    merged['Low_GOLD'], 
    merged['Close_GOLD'], 
    merged['Open_GOLD'])



In [282]:
#C) Volatility features (S&P 500):

#C.1) Forward implied volatility from VIX
#C.1.1) VIX log level
merged['Log_VIX'] = np.log(merged['Close_VIX'])

#C.1.2) VIX log changes
merged['Log_VIX_Change'] = merged['Log_VIX'].diff()

#C.2) Lagged realized volatility:

#c.2.1) Daily laged realized volatility:
merged['Log_RV_Daily'] = np.log(merged['rv5'])

# 2. Weekly Average Lag (5 Days)
# Captures the short-term regime.
merged['Log_RV_Weekly'] = merged['Log_RV_Daily'].rolling(window=5).mean()

# 3. Monthly Average Lag (22 Days)
# Captures the medium-term regime.
merged['Log_RV_Monthly'] = merged['Log_RV_Daily'].rolling(window=22).mean()


In [283]:
#the target: 

merged['Target_Log_RV'] = merged['Log_RV_Daily'].shift(-1)

In [ ]:
#D) Momentum and Distribution: 


# 1. ROLLING KURTOSIS (Distribution)
merged['Kurtosis_60D'] = merged['ret_sp'].rolling(window=60).kurt()


# 2. MOMENTUM (Trend)

# A) Daily (T): This is just the daily log return (Already calculated)
merged["Momentum_1D_sp"] = merged['ret_sp']

# B) Weekly (5 Days): Short-term sentiment
merged['Momentum_1W_sp'] = merged['ret_sp'].rolling(window=5).sum()

# C) Monthly (22 Days): Medium-term trend
merged['Momentum_1M_sp'] = merged['ret_sp'].rolling(window=22).sum()


# 3. RELATIVE STRENGTH INDEX (RSI 14)

def calculate_rsi(series, window=14):
    delta = series.diff()
    
    # Separate Gains and Losses
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    # Wilder's Smoothing (Standard RSI method)
    avg_gain = gain.ewm(com=window-1, min_periods=window).mean()
    avg_loss = loss.ewm(com=window-1, min_periods=window).mean()
    
    # Calculation
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

merged['RSI_14'] = calculate_rsi(merged['Close_S&P'], window=14)

merged.drop(columns = ["ret_sp"], inplace=True)



In [ ]:
#  Set the option permanently for this session
pd.set_option('display.max_columns', None)


merged.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,DGS10,Relative_Volume,True_Range_S&P,ret_gold,ret_oil,diff_dgs10,Corr_SP_Gold,Corr_SP_Oil,Corr_SP_DGS10,True_Range_Oil,True_Range_Gold,Vol_GK_Oil,Vol_GK_Gold,Log_VIX,Log_VIX_Change,Log_RV_Daily,Log_RV_Weekly,Log_RV_Monthly,Target_Log_RV,Kurtosis_60D,Momentum_1D_sp,Momentum_1W_sp,Momentum_1M_sp,RSI_14
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.270020,1320.280029,1276.050049,1320.280029,1129400000,29.990000,30.400000,27.920000,27.930000,268.399994,268.399994,268.399994,268.399994,27.200001,27.400000,26.600000,27.250000,0.014231,0.0492,1.000000,44.229980,NaN,NaN,NaN,NaN,NaN,NaN,0.799999,0.000000,0.020922,0.0,3.400864,NaN,-4.252367,NaN,NaN,-3.452599,NaN,NaN,NaN,NaN,NaN
2001-01-03,1347.560059,1347.760010,1274.619995,1283.270020,1880700000,26.600000,29.959999,25.990000,29.629999,268.000000,268.000000,268.000000,268.000000,27.950001,28.139999,27.049999,27.230000,0.031663,0.0514,1.249593,73.140015,-0.001491,0.027200,0.043745,NaN,NaN,NaN,1.090000,0.399994,0.022742,0.0,3.280911,-0.119953,-3.452599,NaN,NaN,-4.373293,NaN,0.048884,NaN,NaN,NaN
2001-01-04,1333.339966,1350.239990,1329.140015,1347.560059,2131000000,26.969999,27.190001,26.129999,26.350000,267.299988,267.299988,267.299988,267.299988,28.200001,28.780001,27.850000,28.200001,0.012610,0.0503,1.243508,21.099976,-0.002615,0.008905,-0.021633,NaN,NaN,NaN,0.930000,0.700012,0.023227,0.0,3.294725,0.013814,-4.373293,NaN,NaN,-4.158793,NaN,-0.010609,NaN,NaN,NaN
2001-01-05,1298.349976,1334.770020,1294.949951,1333.339966,1430800000,28.670000,29.040001,27.209999,27.809999,268.000000,268.000000,268.000000,268.000000,28.000000,28.799999,27.799999,28.150000,0.015626,0.0493,0.870859,39.820068,0.002615,-0.007117,-0.020081,NaN,NaN,NaN,1.000000,0.700012,0.024767,0.0,3.355851,0.061126,-4.158793,NaN,NaN,-4.701309,NaN,-0.026593,NaN,NaN,NaN
2001-01-08,1295.859985,1298.349976,1276.290039,1298.349976,1115500000,29.840000,30.799999,29.719999,29.980000,268.000000,268.000000,268.000000,268.000000,27.350000,28.400000,27.150000,28.200001,0.009083,0.0494,0.725538,22.059937,0.000000,-0.023488,0.002026,NaN,NaN,NaN,1.250000,0.000000,0.025519,0.0,3.395850,0.039998,-4.701309,-4.187672,NaN,-4.376033,NaN,-0.001920,NaN,NaN,NaN


In [286]:
#drop the 60 firsst row to avoid Nan values do to rolling correlations and moving averages
merged = merged.iloc[60: , :]

In [287]:
#Drop the last line due to the shift to create the target variable
merged = merged.iloc[:-1 , :]

In [288]:
merged

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,DGS10,Relative_Volume,True_Range_S&P,ret_gold,ret_oil,diff_dgs10,Corr_SP_Gold,Corr_SP_Oil,Corr_SP_DGS10,True_Range_Oil,True_Range_Gold,Vol_GK_Oil,Vol_GK_Gold,Log_VIX,Log_VIX_Change,Log_RV_Daily,Log_RV_Weekly,Log_RV_Monthly,Target_Log_RV,Kurtosis_60D,Momentum_1D_sp,Momentum_1W_sp,Momentum_1M_sp,RSI_14
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-04-02,1145.869995,1169.510010,1137.510010,1160.329956,1254900000,31.209999,31.610001,29.160000,29.400000,255.600006,257.799988,255.000000,257.200012,25.590000,26.469999,25.549999,26.350000,0.003910,0.0498,1.004645,32.000000,-0.008958,-0.031162,0.010091,-0.077436,0.147735,0.486520,0.920000,2.899994,0.017170,0.006677,3.440739,0.085934,-5.544153,-4.487533,-4.254208,-4.108424,1.384399,-0.012540,-0.005934,-0.093311,38.830397
2001-04-03,1106.459961,1145.869995,1100.189941,1145.869995,1386100000,34.720001,35.200001,32.919998,32.919998,256.799988,258.600006,255.500000,255.500000,26.190001,26.280001,25.490000,25.700001,0.016434,0.0494,1.117052,45.680054,0.004684,0.023176,-0.008065,-0.101200,0.052936,0.375447,0.790001,3.100006,0.018111,0.007923,3.547316,0.106577,-4.108424,-4.411001,-4.249163,-3.957253,-0.000227,-0.034998,-0.066186,-0.113897,33.139865
2001-04-04,1103.250000,1117.500000,1091.989990,1106.459961,1425590000,34.070000,34.740002,33.209999,34.279999,258.399994,260.500000,258.000000,258.299988,27.120001,27.240000,26.129999,26.650000,0.019116,0.0494,1.159867,25.510010,0.006211,0.034894,0.000000,-0.102632,0.056083,0.370781,1.110001,3.700012,0.027337,0.006815,3.528417,-0.018899,-3.957253,-4.389214,-4.242246,-4.085195,0.046944,-0.002905,-0.044358,-0.117842,32.719272
2001-04-05,1151.439941,1151.469971,1103.250000,1103.250000,1368000000,29.940001,32.509998,29.879999,32.509998,258.100006,259.100006,257.700012,259.100006,27.260000,27.780001,26.650000,27.100000,0.016820,0.0498,1.113960,48.219971,-0.001162,0.005149,0.008065,-0.090352,0.057346,0.346696,1.130001,1.399994,0.029135,0.002983,3.399195,-0.129222,-4.085195,-4.374130,-4.242657,-3.902031,0.848875,0.042753,0.003036,-0.069394,44.174101
2001-04-06,1128.430054,1151.439941,1119.290039,1151.439941,1266800000,31.690001,32.040001,30.219999,30.389999,260.000000,260.200012,257.799988,259.500000,27.059999,27.280001,26.809999,26.900000,0.020201,0.0489,1.029439,32.149902,0.007335,-0.007364,-0.018238,-0.106825,0.062777,0.366342,0.470001,2.400024,0.011723,0.006442,3.456001,0.056806,-3.902031,-4.319411,-4.205430,-4.401230,0.689501,-0.020186,-0.027877,-0.095421,40.618111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-09,3911.229980,3918.350098,3902.639893,3910.489990,4568320000,21.629999,22.260000,20.650000,21.570000,1835.300049,1843.300049,1830.699951,1841.900024,58.360001,58.619999,57.270000,58.110001,0.003064,0.0118,0.924728,15.710205,0.001854,0.006705,-0.008439,-0.057554,0.297631,0.293174,1.349998,12.600098,0.016257,0.004306,3.074081,0.018195,-5.788097,-5.409578,-5.028135,-4.729817,1.085521,-0.001114,0.021951,0.042592,64.032937
2021-02-10,3909.879883,3931.500000,3884.939941,3920.780029,4837070000,21.990000,23.850000,19.690001,21.639999,1840.599976,1851.099976,1837.000000,1842.599976,58.680000,58.910000,58.080002,58.450001,0.008828,0.0115,0.978463,46.560059,0.002884,0.005468,-0.025752,-0.050895,0.301848,0.296838,0.829998,15.799927,0.009732,0.005364,3.090588,0.016507,-4.729817,-5.325359,-5.017457,-5.027483,1.103273,-0.000345,0.020597,0.027509,63.805239
2021-02-11,3916.379883,3925.989990,3890.389893,3916.399902,4590960000,21.250000,23.250000,21.110001,22.090000,1824.900024,1845.000000,1820.900024,1840.500000,58.240002,58.709999,57.840000,58.400002,0.006555,0.0116,0.929678,35.600098,-0.008566,-0.007527,0.008658,-0.037798,0.285508,0.247044,0.869999,24.099976,0.

In [289]:
#Nan check:
merged.isna().sum().sum()

np.int64(0)

In [ ]:
#Get rid of useless collumn: 
raw_keywords = ['Open', 'High', 'Low', 'Close', 'Volume_S&P']

# List comprehension to find the exact column names in your dataframe
cols_to_drop = [c for c in merged.columns if any(k in c for k in raw_keywords)]

print(cols_to_drop)

['Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL']


In [291]:
print(merged.columns.tolist())

['Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL', 'rv5', 'DGS10', 'Relative_Volume', 'True_Range_S&P', 'ret_gold', 'ret_oil', 'diff_dgs10', 'Corr_SP_Gold', 'Corr_SP_Oil', 'Corr_SP_DGS10', 'True_Range_Oil', 'True_Range_Gold', 'Vol_GK_Oil', 'Vol_GK_Gold', 'Log_VIX', 'Log_VIX_Change', 'Log_RV_Daily', 'Log_RV_Weekly', 'Log_RV_Monthly', 'Target_Log_RV', 'Kurtosis_60D', 'Momentum_1D_sp', 'Momentum_1W_sp', 'Momentum_1M_sp', 'RSI_14']


In [292]:
merged.drop(columns=cols_to_drop, inplace=True)
merged.drop(columns = ["DGS10"], inplace=True)
merged.drop(columns = ["rv5"], inplace=True)

In [293]:
print(merged.columns.tolist())

['Relative_Volume', 'True_Range_S&P', 'ret_gold', 'ret_oil', 'diff_dgs10', 'Corr_SP_Gold', 'Corr_SP_Oil', 'Corr_SP_DGS10', 'True_Range_Oil', 'True_Range_Gold', 'Vol_GK_Oil', 'Vol_GK_Gold', 'Log_VIX', 'Log_VIX_Change', 'Log_RV_Daily', 'Log_RV_Weekly', 'Log_RV_Monthly', 'Target_Log_RV', 'Kurtosis_60D', 'Momentum_1D_sp', 'Momentum_1W_sp', 'Momentum_1M_sp', 'RSI_14']


In [294]:
#Some rename for clarity:
rename_mapping = {
    'ret_gold': 'Momentum_1D_gold',
    'ret_oil':  'Momentum_1D_oil',
    
     'diff_dgs10': 'Momentum_1D_yield' 
}

merged.rename(columns=rename_mapping, inplace=True)

In [295]:
print(merged.columns.tolist())

['Relative_Volume', 'True_Range_S&P', 'Momentum_1D_gold', 'Momentum_1D_oil', 'Momentum_1D_yield', 'Corr_SP_Gold', 'Corr_SP_Oil', 'Corr_SP_DGS10', 'True_Range_Oil', 'True_Range_Gold', 'Vol_GK_Oil', 'Vol_GK_Gold', 'Log_VIX', 'Log_VIX_Change', 'Log_RV_Daily', 'Log_RV_Weekly', 'Log_RV_Monthly', 'Target_Log_RV', 'Kurtosis_60D', 'Momentum_1D_sp', 'Momentum_1W_sp', 'Momentum_1M_sp', 'RSI_14']


In [296]:

# index=True ensures the Date index is preserved
merged.to_csv(os.path.join(DATA_DIR, 'merged_features.csv'), index=True)

